In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from pathlib import Path
import os
import sys

C:\Users\Acer\PycharmProjects\sheduling_agent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should show your GPU name
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
root_path = Path(os.getcwd().split('application_source')[0] + 'application_source')
print(root_path)

True
NVIDIA GeForce GTX 1660 Ti
cuda
C:\Users\Acer\PycharmProjects\sheduling_agent\application_source


In [5]:
model_id = Path.joinpath(root_path, "models/Qwen/Qwen2.5-3B-Instruct")
model_id

WindowsPath('C:/Users/Acer/PycharmProjects/sheduling_agent/application_source/models/Qwen/Qwen2.5-3B-Instruct')

In [6]:
# model_id = Path("models\\Qwen\\Qwen2.5-3B-Instruct")
model_id = Path.joinpath(root_path, "models/Qwen/Qwen2.5-3B-Instruct")

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # <-- match input dtype
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4"  # or "fp4" if needed
)

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map="auto",
                                             trust_remote_code=True)
model.eval()

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 3/3 [00:17<00:00,  5.95s/it]


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=True)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((2048,), 

In [7]:
print(model.hf_device_map)

{'': 0}


In [5]:
def forward(context):

    text = tokenizer.apply_chat_template(
    context,
    tokenize=False,
    add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=8000,                  # <-- use greedy decoding (not beam search)
        temperature=0.001,              # <-- has no effect if do_sample=False
        top_p=1.0,
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [6]:
def query_agent(query):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": query}
    ]
    return messages

In [7]:
query = "Generate a meeting recap transcript beetween an advisor and customr where advisor is trying to sell 3 stock related products, genrate at max 10 conversation withing a session and advisor conviences cusomer to puchase at least one return conersation only"

res = forward(query_agent(query))
print(res)

**Advisor:** Hi John, it's great to see you again. I hope you're doing well.

**Customer:** Hi, yes, I am. How are you?

**Advisor:** I'm doing well, thanks. I was wondering if you had a chance to review the reports we sent out last week? We have some new products that might interest you.

**Customer:** Sure, I did look at them. What are they?

**Advisor:** Well, we have three different stock-related products that could potentially benefit your portfolio. Let’s go through them quickly.

**Customer:** Okay, let’s do it.

**Advisor:** Great. The first one is our Growth Accelerator Package. It includes a detailed analysis of five high-growth stocks, along with a customized investment strategy. This package can help you identify and capitalize on emerging trends in the market.

**Customer:** That sounds interesting. What kind of trends does it cover?

**Advisor:** It covers a variety of sectors including technology, healthcare, and renewable energy. We analyze the latest news, financials, 

In [ ]:
prompt = "Generate a meeting recap transcript beetween an advisor and customr where advisor is trying to sell 3 stock related products, genrate at max 10 conversation withing a session and advisor conviences cusomer to puchase at least one return conersation only"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model.generate(**inputs, do_sample=False, max_new_tokens=8000)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
print(response)

In [ ]:
print("""\nHere\'s a structured format for the conversation between the advisor and the customer:\n\n**Customer:** [Name]  \n**Advisor:** Trinh\n\n---\n\n**Customer:** Hi Trinh, how are you today?\n\n**Advisor Trinh:** Hi [Name], I\'m doing great, thanks! How are you?\n\n**Customer:** I\'m good, thanks. So, Trinh, what can you do for me today?\n\n**Advisor Trinh: **Great question! Let\'s start with our first product - the "Daily Gain" Stock Alert. This is a daily email that highlights stocks showing strong positive momentum. It\'s perfect for those looking to stay ahead of the market.\n\n**Customer:** That sounds interesting. What does it include?\n\n**Advisor Trinh:** Well, it includes the top 5 stocks with the highest positive momentum, along with a brief analysis of why they\'re performing well. You\'ll also get tips on how to capitalize on this momentum in your portfolio.\n\n**Customer:** Okay, that seems useful. How much is it?\n\n**Advisor Trinh:** The subscription fee is $29 per month. But if you sign up now, we can offer you a special introductory price of $19 for the first month.\n\n**Customer:** Hmm, that\'s not bad. How often will I receive these alerts?\n\n**Advisor Trinh:** Once a day, right to your inbox. And remember, you can unsubscribe anytime if you change your mind.\n\n**Customer:** Alright, I\'ll give it a try. What\'s next?\n\n**Advisor Trinh:** Great! Our second product is the "Market Pulse" Stock Report. This is a weekly report that provides detailed analysis of key sectors and stocks, helping you make informed investment decisions.\n\n**Customer:** Sounds like it could be helpful. How much is it?\n\n**Advisor Trinh:** For the full year, it\'s $49. Again, we can offer you a special introductory price of $39 for the first year.\n\n**Customer:** That\'s still better than the other options. How often will I receive these reports?\n\n**Advisor Trinh:** Weekly emails sent directly to your inbox, covering all the important sectors and stocks.\n\n**Customer:** Alright, I\'ll consider it. What\'s the last option?\n\n**Advisor Trinh:** Our final product is the "Stock Master" Portfolio Management Service. This service offers personalized stock recommendations based on your investment goals and risk tolerance.\n\n**Customer:** That sounds really comprehensive. How much is it?\n\n**Advisor Trinh:** For the full year, it\'s $99. We can offer you a special introductory price of $79 for the first year.\n\n**Customer:** Wow, that\'s quite a bit more expensive. Is there any way to get a discount?\n\n**Advisor Trinh:** Absolutely! If you commit to signing up for the full year, we can offer you a bulk discount of $10 off the annual price.\n\n**Customer:** Alright, I\'ll think about it. Thanks for explaining everything so clearly.\n\n**Advisor Trinh:** Of course, [Name]. Remember, you can always reach out if you have any questions or need further assistance. I look forward to hearing from you soon.\n\n**Customer:** Sure thing, Trinh. Talk to you later.\n\n**Advisor Trinh:** Definitely, [Name]. Have a great day!\n\n---\n\nAfter several rounds of discussion, if Trinh convinces the customer to purchase at least one of the three products, the conversation would continue as follows:\n\n**Customer:** Trinh, I\'m leaning towards the "Daily Gain" Stock Alert. Can you show me some examples of the stocks included?\n\n**Advisor Trinh:** Absolutely, [Name]. Here are a few examples from the past week:\n- **Company A**: Showing strong growth in earnings and revenue, with potential for further expansion.\n- **Company B**: High liquidity and low volatility, making it a solid addition to your portfolio.\n- **Company C**: Rising dividends and increasing market share, indicating strong future potential.\n\n**Customer:** Those sound promising. How do I sign up?\n\n**Advisor Trinh:** Simply click on the link provided in the email, and you\'ll be taken through the subscription process. If you have any issues, feel free to reach out to our support team.\n\n**Customer:** Perfect, thank you, Trinh.\n\n**Advisor Trinh:** You\'re welcome, [Name]. If you decide to go with the "Daily Gain" Stock Alert, I\'ll send you a confirmation email shortly.\n\n**Customer:** Great, thanks again.\n\n**Advisor Trinh:** You\'re welcome. If you have any other questions or need further assistance, don\'t hesitate to reach out.\n\n**Customer:** Will do, Trinh. Talk to you soon.\n\n**Advisor Trinh:** Definitely, [Name]. Have a great day!  \n**Customer:** You too, Trinh.\n\n---\n\nIf the customer decides to purchase, the conversation would then proceed with the subscription confirmation and any additional details needed for setup. If the customer declines, the conversation would end with a polite goodbye.'""")

In [ ]:
import sys

In [ ]:
prompt = "Generate a conversation beetween an advisor and customr where advisor is trinh to sell 3 stock related products, geerate at least 10 conversation withing a session and advisor conviences cusomer to puchase at least one"
input_ids = tokenizer(prompt, return_tensors="pt").to(device).input_ids

# Generation settings
max_new_tokens = 8000
stop_token = tokenizer.eos_token_id

# Streaming loop
print("\nResponse:\n")
generated = input_ids
for _ in range(max_new_tokens):
    with torch.no_grad():
        outputs = model(generated, do_sample=False)
        logits = outputs.logits[:, -1, :]
        next_token_id = torch.argmax(logits, dim=-1).unsqueeze(0)

    generated = torch.cat([generated, next_token_id], dim=1)

    # Decode and print the newly generated token
    new_token = tokenizer.decode(next_token_id[0])
    if next_token_id[0].item() == 151643:
        break
    sys.stdout.write(new_token)
    sys.stdout.flush()


print()

In [ ]:
stop_token

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame({"id":tokenizer.all_special_ids, "token":tokenizer.all_special_tokens})